In [ ]:
import functions as functions
import numpy as np

In [ ]:
elements = functions.import_elements('../Data/Elements.xlsx')

In [ ]:
paths = []
for _, elem in elements[elements["type"]=="customer"].iterrows():
    p = functions.astar(elements, elem.squeeze(), num_paths=4, D=20)
    paths.extend(p)

for p in paths:
    print(p.get_names())
print(f"Found {len(paths)} paths")

In [ ]:
H = functions.BuildHMatrix(paths, elements)

In [ ]:
C = functions.Subset(elements, ["customer"])
T = functions.Subset(elements, ["junction"])
R = functions.Subset(elements, ["line"])

Hc, Hr, Ht = functions.MatrixDivision(elements, H, C,R,T)

# Optimization problem

In [ ]:
model, Tr, P = functions.optimization_problem(C,R,T, H,Hc,Hr,Ht)

# Results explanation

## P & Tr

In [ ]:
Tr_sol, P_sol, = functions.transform_matrices_in_dfs(H,T,R, Tr, P)

In [ ]:
estimated_optimal_paths = [p for p in P_sol.columns.values if P_sol[p][0] == 1]
estimated_optimal_paths

In [ ]:
H

In [ ]:
Tr_sol

## First constrain


### left-hand side

In [ ]:
np.sum(Hr.values,axis=1)

In [ ]:
np.sum(Hr.values,axis=1) * P_sol.values

In [ ]:
np.sum(Hr.values,axis=1) * P_sol.values * np.transpose(Ht.values)

### rigth-hand side

In [ ]:
(Tr_sol.values @ np.transpose(Hr.values))

In [ ]:
(Tr_sol.values @ np.transpose(Hr.values)) * np.transpose(Ht.values) * P_sol.values

## Second constraint

In [ ]:
P_sol.values @ Hc.values

## Third constraint

In [ ]:
sum(Tr_sol.values)